In [1]:
# Setup: Import all the required modules
import sys
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Import the HRM-EEG modules
try:
    from eeg_hierarchical_processor import HierarchicalEEGProcessor, ModelComparator, ModelResults
    from attention_hierarchical_processor import HRMEEGProcessor
    print("✅ Core HRM-EEG modules imported successfully!")
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("💡 Make sure you're in the HRM-EEG repository directory")
    print("💡 Run the 0_setup.ipynb notebook first to clone the repository")

# Configure matplotlib for better plots
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("🚀 Setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


❌ Import failed: No module named 'eeg_hierarchical_processor'
💡 Make sure you're in the HRM-EEG repository directory
💡 Run the 0_setup.ipynb notebook first to clone the repository
🚀 Setup complete!
PyTorch version: 2.7.1+cu126
Using device: cpu


In [2]:
# Create the hierarchical EEG processor
print("🧠 Creating Hierarchical EEG Processor...")

# Initialize the processor
processor = HierarchicalEEGProcessor()
print("✅ Processor created successfully!")

# Generate synthetic hierarchical data
print("\n📊 Generating synthetic hierarchical EEG data...")

# Check the correct method signature first
print("📋 Available methods in HierarchicalEEGProcessor:")
methods = [method for method in dir(processor) if not method.startswith('_')]
for method in methods[:10]:  # Show first 10 methods
    print(f"   • {method}")

# Generate synthetic data using the correct method signature
try:
    # Try different possible method signatures
    X, y = processor.generate_synthetic_data()
    print(f"✅ Synthetic data generated!")
    print(f"   📊 Data shape: {X.shape}")
    print(f"   📊 Labels shape: {y.shape}")
    print(f"   📊 Label distribution: {np.bincount(y.astype(int))}")
except Exception as e:
    print(f"⚠️  First attempt failed: {e}")
    # Try to inspect the method signature
    import inspect
    try:
        sig = inspect.signature(processor.generate_synthetic_data)
        print(f"📋 Method signature: generate_synthetic_data{sig}")
        
        # Try with default parameters
        X, y = processor.generate_synthetic_data(n_samples=1000, n_timepoints=500)
        print(f"✅ Synthetic data generated with explicit parameters!")
        print(f"   📊 Data shape: {X.shape}")
        print(f"   📊 Labels shape: {y.shape}")
        print(f"   📊 Label distribution: {np.bincount(y.astype(int))}")
    except Exception as e2:
        print(f"❌ Could not generate synthetic data: {e2}")
        print("💡 Let's explore what's available in the processor...")
        print(f"💡 Processor type: {type(processor)}")


🧠 Creating Hierarchical EEG Processor...


NameError: name 'HierarchicalEEGProcessor' is not defined

In [ ]:
# Visualize the hierarchical structure (if data was generated successfully)
if 'X' in locals() and 'y' in locals():
    print("📊 Visualizing hierarchical EEG data structure...")
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Sample signals for each class
    axes[0, 0].set_title("🧠 Raw EEG Signals by Class")
    for class_idx in np.unique(y):
        class_samples = X[y == class_idx]
        if len(class_samples) > 0:
            # Plot first sample of each class
            sample = class_samples[0]
            axes[0, 0].plot(sample[:200], label=f'Class {int(class_idx)}', alpha=0.7)
    axes[0, 0].set_xlabel('Time Points')
    axes[0, 0].set_ylabel('Signal Amplitude')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Signal statistics
    axes[0, 1].set_title("📈 Signal Statistics by Class")
    class_means = [np.mean(X[y == class_idx]) for class_idx in np.unique(y)]
    class_stds = [np.std(X[y == class_idx]) for class_idx in np.unique(y)]
    x_pos = range(len(class_means))
    axes[0, 1].bar([x - 0.2 for x in x_pos], class_means, width=0.4, label='Mean', alpha=0.7)
    axes[0, 1].bar([x + 0.2 for x in x_pos], class_stds, width=0.4, label='Std Dev', alpha=0.7)
    axes[0, 1].set_xlabel('Class')
    axes[0, 1].set_ylabel('Amplitude')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Class distribution
    axes[1, 0].set_title("🎯 Class Distribution")
    class_counts = np.bincount(y.astype(int))
    axes[1, 0].bar(range(len(class_counts)), class_counts, alpha=0.7)
    axes[1, 0].set_xlabel('Class')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Signal complexity over time
    axes[1, 1].set_title("🔄 Signal Complexity Analysis")
    # Calculate rolling standard deviation as a measure of complexity
    sample_signal = X[0]  # First sample
    window_size = 50
    rolling_std = pd.Series(sample_signal).rolling(window=window_size).std()
    axes[1, 1].plot(rolling_std, color='purple', alpha=0.8)
    axes[1, 1].set_xlabel('Time Points')
    axes[1, 1].set_ylabel('Rolling Std Dev')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('🧠 Hierarchical EEG Data Exploration', y=1.02, fontsize=16)
    plt.show()
    
    print("✅ Visualization complete!")
    print(f"💡 The data shows {len(np.unique(y))} classes with hierarchical structure")
    
else:
    print("⚠️  No data available for visualization")
    print("💡 Try running the data generation cell first")
